# What Am I Reading?

In [ ]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import struct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#### 1. Get Data

In [ ]:
DATASET_PATH = "./NeuralNews/dataset"

In [ ]:
class newsDataset(Dataset):

    def __init__(self, path, split_label, caption_json):
        